In [1]:

import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anjin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

df = pd.read_csv(r"C:\Users\anjin\Downloads\reviews_badminton\data.csv")
df.head()


Reviewer Name               Review Title  \
0            Kamal Suresh               Nice product   
1       Flipkart Customer     Don't waste your money   
2  A. S. Raja Srinivasan   Did not meet expectations   
3     Suresh Narayanasamy                       Fair   
4               ASHIK P A                Over priced   

               Place of Review  Up Votes  Down Votes     Month  \
0   Certified Buyer, Chirakkal     889.0        64.0  Feb 2021   
1   Certified Buyer, Hyderabad     109.0         6.0  Feb 2021   
2  Certified Buyer, Dharmapuri      42.0         3.0  Apr 2021   
3     Certified Buyer, Chennai      25.0         1.0       NaN   
4                          NaN     147.0        24.0  Apr 2016   

                                         Review text  Ratings  
0  Nice product, good quality, but price is now r...        4  
1  They didn't supplied Yonex Mavis 350. Outside ...        1  
2  Worst product. Damaged shuttlecocks packed in ...        1  
3  Quite O. K. , but nowadays  the quality of the...        3  
4  Over pricedJust â?¹620 ..from retailer.I didn'...        1

In [3]:

df = df.dropna(subset=['Review text'])


In [4]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return ' '.join(words)

df['clean_review'] = df['Review text'].apply(clean_text)


In [5]:

def map_sentiment(r):
    if r <= 2:
        return 0   # Negative
    elif r == 3:
        return 1   # Neutral
    else:
        return 2   # Positive

df['sentiment'] = df['Ratings'].apply(map_sentiment)
df['sentiment'].value_counts()


sentiment
2    6823
0    1072
1     615
Name: count, dtype: int64

In [6]:

X = df['clean_review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [7]:

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [9]:

param_grid = {
    "C": [0.01, 0.1, 1, 10]
}


lr = LogisticRegression(
    solver='lbfgs',
    class_weight='balanced',
    max_iter=1000
)



grid = GridSearchCV(
    lr,
    param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1
)

grid.fit(X_train_tfidf, y_train)
lr_best = grid.best_estimator_


In [10]:

y_pred_lr = lr_best.predict(X_test_tfidf)

print(classification_report(
    y_test, y_pred_lr,
    target_names=['Negative', 'Neutral', 'Positive']
))

confusion_matrix(y_test, y_pred_lr)


              precision    recall  f1-score   support

    Negative       0.57      0.67      0.61       214
     Neutral       0.13      0.46      0.21       123
    Positive       0.95      0.71      0.81      1365

    accuracy                           0.69      1702
   macro avg       0.55      0.61      0.54      1702
weighted avg       0.84      0.69      0.74      1702



array([[143,  50,  21],
       [ 33,  57,  33],
       [ 76, 318, 971]], dtype=int64)

In [11]:

svm = LinearSVC(class_weight='balanced')
svm.fit(X_train_tfidf, y_train)

y_pred_svm = svm.predict(X_test_tfidf)

print(classification_report(
    y_test, y_pred_svm,
    target_names=['Negative', 'Neutral', 'Positive']
))


              precision    recall  f1-score   support

    Negative       0.62      0.63      0.63       214
     Neutral       0.28      0.23      0.25       123
    Positive       0.92      0.93      0.92      1365

    accuracy                           0.84      1702
   macro avg       0.61      0.60      0.60      1702
weighted avg       0.83      0.84      0.84      1702



In [12]:


text = 'It is good but I will give 2.5/5'
vec = tfidf.transform([text])

print('Logistic Regression:', lr_best.predict(vec))
print('Linear SVM:', svm.predict(vec))


Logistic Regression: [2]
Linear SVM: [2]


In [13]:
import pickle

pickle.dump(lr_best, open("sentiment_model_3class.pkl", "wb"))
pickle.dump(tfidf, open("tfidf_3class.pkl", "wb"))


In [14]:
df['Review text']

0       Nice product, good quality, but price is now r...
1       They didn't supplied Yonex Mavis 350. Outside ...
2       Worst product. Damaged shuttlecocks packed in ...
3       Quite O. K. , but nowadays  the quality of the...
4       Over pricedJust â?¹620 ..from retailer.I didn'...
                              ...                        
8505    Delivered before time but price is high from m...
8506    up to the mark but same is available in market...
8507                         Nice delivery speedREAD MORE
8508    No complaints about the item . Its the best on...
8509    Not sure why we have charged for this product ...
Name: Review text, Length: 8510, dtype: object

In [15]:
import streamlit as st

st.title("Flipkart Review Sentiment Analysis")

text = st.text_area("Enter Review")

if st.button("Predict"):
    pred = model.predict(tfidf.transform([text]))
    st.success(f"Sentiment: {pred[0]}")


2026-02-03 20:00:55.254 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-03 20:00:55.362 
  command:

    streamlit run C:\Users\anjin\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-02-03 20:00:55.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-03 20:00:55.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-03 20:00:55.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-03 20:00:55.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-03 20:00:55.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-03 20:00:55.365 Thread 'MainThread': mi